In [1]:
import torch.nn as nn
import torch
from torchvision import transforms
from torch.utils.data import DataLoader,Dataset
import torch.nn.functional as F
import torch.optim as optim
import os
import gzip
import numpy as np

In [2]:
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock,self).__init__()
        self.channels = channels
        
        self.conv1 = nn.Conv2d(channels,channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(channels,channels, kernel_size=3, padding=1)
        
    def forward(self, x):
        y = F.relu(self.conv1(x))
        y = self.conv2(y)
        
        return F.relu(x+y)

In [3]:
class ResNet(nn.Module):
    def __init__(self):
        super(ResNet,self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        
        self.rblock1 = ResidualBlock(16)
        self.rblock2 = ResidualBlock(32)
        
        self.fc = nn.Linear(512, 10)
        
    def forward(self, x):
        in_size = x.size(0)
        x = self.mp(F.relu(self.conv1(x)))
        x = self.rblock1(x)
        x = self.mp(F.relu(self.conv2(x)))
        x = self.rblock2(x)
        x = x.view(in_size, -1)
        x = self.fc(x)
        return x

In [4]:
# 构建数据集
class MNISTDataset(Dataset):
    def __init__(self, root, train=True, transform=None):
        self.root = root
        self.train = train
        self.transform = transform

        if self.train:
            images_file = 'train-images-idx3-ubyte.gz'
            labels_file = 'train-labels-idx1-ubyte.gz'
        else:
            images_file = 't10k-images-idx3-ubyte.gz'
            labels_file = 't10k-labels-idx1-ubyte.gz'

        self.images_path = os.path.join(root, images_file)
        self.labels_path = os.path.join(root, labels_file)

        self.images, self.labels = self.load_data()

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = int(self.labels[idx])

        if self.transform:
            image = self.transform(image)

        return image, label

    def load_data(self):
        with gzip.open(self.images_path, 'rb') as f_images:
            images = np.frombuffer(f_images.read(), dtype=np.uint8, offset=16).reshape(-1, 28, 28)

        with gzip.open(self.labels_path, 'rb') as f_labels:
            labels = np.frombuffer(f_labels.read(), dtype=np.uint8, offset=8)

        return images, labels

In [5]:
batch_size = 64
transform = transforms.Compose([
transforms.ToTensor(),
transforms.Normalize((0.1307, ), (0.3081, ))
])

In [6]:
train_dataset = MNISTDataset(root='./mnist_dataset/', train=True, transform=transform)
train_loader =  DataLoader(train_dataset,shuffle=True,batch_size=batch_size)

test_dataset = MNISTDataset(root='./mnist_dataset/', train=False, transform=transform)
test_loader =  DataLoader(test_dataset,shuffle=False,batch_size=batch_size)
print(test_dataset.__len__())
print(train_dataset.__len__())

10000
60000


In [7]:
model = ResNet()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNet(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (mp): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (rblock1): ResidualBlock(
    (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (rblock2): ResidualBlock(
    (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (fc): Linear(in_features=512, out_features=10, bias=True)
)

In [8]:
# construct loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [9]:
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader,0):
        inputs, target = data
        inputs, target = inputs.to(device), target.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs,target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if batch_idx%300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, batch_idx + 1, running_loss / 300))
            running_loss = 0.0

In [10]:
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('accuracy on test set: %d %% ' % (100*correct/total))
    return correct/total

In [ ]:
import matplotlib.pyplot as plt

if __name__ == '__main__':
    epoch_list = []
    acc_list = []
    
    for epoch in range(10):
        train(epoch)
        acc = test()
        epoch_list.append(epoch)
        acc_list.append(acc)
    
    plt.plot(epoch_list,acc_list)
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.show()

D:\application\Anaconda\envs\pytorch-gpu\lib\site-packages\torchvision\transforms\functional.py:152: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:212.)
  img = torch.from_numpy(pic.transpose((2, 0, 1))).contiguous()


[1,   300] loss: 0.479
[1,   600] loss: 0.153
[1,   900] loss: 0.109
accuracy on test set: 97 % 
[2,   300] loss: 0.086
[2,   600] loss: 0.080
[2,   900] loss: 0.072
accuracy on test set: 98 % 
[3,   300] loss: 0.060
[3,   600] loss: 0.056
[3,   900] loss: 0.057
accuracy on test set: 98 % 
[4,   300] loss: 0.049
[4,   600] loss: 0.049
[4,   900] loss: 0.045
accuracy on test set: 98 % 
[5,   300] loss: 0.042
[5,   600] loss: 0.040
[5,   900] loss: 0.043
accuracy on test set: 98 % 
[6,   300] loss: 0.035
[6,   600] loss: 0.037
[6,   900] loss: 0.036
accuracy on test set: 98 % 
[7,   300] loss: 0.027
[7,   600] loss: 0.033
[7,   900] loss: 0.034
accuracy on test set: 98 % 
[8,   300] loss: 0.028
[8,   600] loss: 0.026
[8,   900] loss: 0.031
accuracy on test set: 98 % 
[9,   300] loss: 0.024
[9,   600] loss: 0.025
[9,   900] loss: 0.026
